## <span style="color:red">Generating Log Files Randomly</span>


In [1]:
# Import required libraries
import os
import random
from datetime import datetime, timedelta

# Function to generate a log file with random data
def generate_log_file(file_path, num_streams=100, date_str=None):
    # List of countries for random selection
    countries = ["BL", "GN", "BN", "ST", "PW", "HK", "NL", "HM", "NR", "GT", "GU"]
    with open(file_path, 'w') as f:
        for _ in range(num_streams):
            # Generate random song and user IDs
            sng_id = random.randint(10000000, 99999999)
            user_id = random.randint(1000000, 9999999)
            # Randomly select a country from the countries list
            country = random.choice(countries)
            # Write the data to the log file in the format: sng_id | user_id | country
            f.write(f"{sng_id} | {user_id} | {country}\n")

# Function to generate multiple log files with random data
def generate_log_files(folder_path, num_files=40, num_streams=100, start_date=None):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    if start_date is None:
        # If no start_date is provided, use the current date
        start_date = datetime.now()
    else:
        # Parse the start_date string to a datetime object
        start_date = datetime.strptime(start_date, '%Y%m%d')

    for i in range(num_files):
        # Generate date strings for each log file based on the start_date and the number of days
        date_str = (start_date - timedelta(days=i)).strftime('%Y%m%d')
        file_name = f"listen-{date_str}.log"
        file_path = os.path.join(folder_path, file_name)
        # Generate a log file with random data
        generate_log_file(file_path, num_streams=num_streams)

# Example Usage:
# Generate 10 log files, each containing 100 random log entries
generate_log_files("sample_listen_2Generated", num_files=40, num_streams=100)


## <span style="color:red"> Log File Analysis - Top 50 Songs by Country</span>


In [2]:
# Import required libraries
import glob  # For file pattern matching
from collections import defaultdict  # For creating a nested defaultdict

# Function to process a single log file and count song streams per country
def process_log_file(log_file_path):
    # Create a nested defaultdict to store song stream counts for each country
    streams_data = defaultdict(lambda: defaultdict(int))

    with open(log_file_path, 'r') as f:
        for line in f:
            try:
                # Split the line into song_id, user_id, and country using '|' as the separator
                sng_id, user_id, country = line.strip().split('|')
                # Convert song_id and user_id to integers (assuming they are numeric fields)
                sng_id = int(sng_id)
                user_id = int(user_id)
                # Increment the song stream count for the given country and song_id
                streams_data[country][sng_id] += 1
            except (ValueError, IndexError):
                # If there is an error (e.g., missing values or invalid format), skip the row and continue
                continue

    return streams_data

# Function to compute the top 50 songs per country based on their stream counts
def compute_top_50(streams_data):
    top_50 = {}

    for country, song_streams in streams_data.items():
        # Sort songs based on their stream counts in descending order
        sorted_songs = sorted(song_streams.items(), key=lambda x: x[1], reverse=True)
        # Take the top 50 songs with the highest stream counts
        top_50[country] = sorted_songs[:50]

    return top_50

def main():
    # Step 1: Get a List of Log Files for the last 10 days
    log_files_directory = "sample_listen_2Generated"
    log_files = glob.glob(os.path.join(log_files_directory, "listen-*.log"))
    log_files.sort(reverse=True)  # Sort in reverse order to get the latest log files first

    # Consider the last 7 log files if available, otherwise take as many as possible
    log_files = log_files[:7]

    # Step 2: Loop Over Log Files
    streams_data = defaultdict(lambda: defaultdict(int))

    for log_file in log_files:
        # Process each log file and update the streams_data dictionary with song stream counts
        streams_data_current_file = process_log_file(log_file)

        for country, song_streams in streams_data_current_file.items():
            for song, streams in song_streams.items():
                # Update the streams_data dictionary with the stream counts for each song in each country
                streams_data[country][song] += streams

    # Step 3: Compute Top 50 Across Dates
    top_50 = compute_top_50(streams_data)

    # Step 4: Write Output for All Dates
    output_directory = "country_top50/"
    os.makedirs(output_directory, exist_ok=True)  # Create the output directory if it doesn't exist
    today_date_str = datetime.now().strftime('%Y%m%d')
    output_file_path = os.path.join(output_directory, f'country_top50_{today_date_str}.txt')
    with open(output_file_path, 'w') as f:
        for country, songs in top_50.items():
            # Format the output for each country's top 50 songs and write to the output file
            song_strings = [f'{sng_id}:{streams}' for sng_id, streams in songs]
            output_string = f"{country}|{','.join(song_strings)}"
            f.write(output_string + '\n')

if __name__ == "__main__":
    main()
